In [9]:
import pandas as pd
import numpy as np

import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error,accuracy_score,f1_score
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [10]:
df_user_data_train = pd.read_csv("train/user_data.csv")
df_train_submission = pd.read_csv("train/train_submissions.csv")
df_problem_data_train = pd.read_csv("train/problem_data.csv")
df_sample_sub = pd.read_csv("sample_submissions_wbscxqU.csv")
df_test_sub = pd.read_csv("test_submissions_NeDLEvX.csv")

In [11]:
df_problem_data_train.drop(['tags'],axis=1,inplace=True)
df_problem_data_train.points.fillna(0,inplace=True)
df_problem_data_train.level_type.fillna('N',inplace=True)
df_user_data_train.country.fillna('Missing',inplace=True)

def postive_only(x):
    if(x<0):
        return(-1*x)
    else:
        return(x)
    
df_user_data_train['submission_count']=df_user_data_train['submission_count'].map(lambda x: postive_only(x))
df_user_data_train['problem_solved']=df_user_data_train['problem_solved'].map(lambda x: postive_only(x))
df_user_data_train['contribution']=df_user_data_train['contribution'].map(lambda x: postive_only(x))
df_user_data_train['follower_count']=df_user_data_train['follower_count'].map(lambda x: postive_only(x))
df_user_data_train['last_online_time_seconds']=df_user_data_train['last_online_time_seconds'].map(lambda x: postive_only(x))
df_user_data_train['max_rating']=df_user_data_train['max_rating'].map(lambda x: postive_only(x))
df_user_data_train['registration_time_seconds']=df_user_data_train['registration_time_seconds'].map(lambda x: postive_only(x))

In [12]:
df_train  = pd.merge(df_train_submission,df_user_data_train, how='left', on='user_id')
df_train = pd.merge(df_train,df_problem_data_train,how='left',on='problem_id')

In [13]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_train['country'] = lb.fit_transform(df_train['country'])
df_train['rank'] = lb.fit_transform(df_train['rank'])
df_train['level_type'] = lb.fit_transform(df_train['level_type'])

In [14]:
X = df_train.drop(['attempts_range','user_id','problem_id'],axis=1).values
y = df_train.attempts_range.values

In [15]:
start_time = time.time()

clf_KN_L1 = KNeighborsClassifier()

y_pred_KN_L1 = cross_val_predict(clf_KN_L1,X,y,cv=8,n_jobs=-1)
print("Training data for next level Created")

print("--- %s seconds ---" % (time.time() - start_time))
print(f1_score(y,y_pred_KN_L1,average="weighted"))

Training data for next level Created
--- 1.34192490578 seconds ---
0.439461902206


In [16]:
df_test  = pd.merge(df_test_sub,df_user_data_train, how='left', on='user_id')
df_test = pd.merge(df_test,df_problem_data_train,how='left',on='problem_id')

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66555 entries, 0 to 66554
Data columns (total 15 columns):
ID                           66555 non-null object
user_id                      66555 non-null object
problem_id                   66555 non-null object
submission_count             66555 non-null int64
problem_solved               66555 non-null int64
contribution                 66555 non-null int64
country                      66555 non-null object
follower_count               66555 non-null int64
last_online_time_seconds     66555 non-null int64
max_rating                   66555 non-null float64
rating                       66555 non-null float64
rank                         66555 non-null object
registration_time_seconds    66555 non-null int64
level_type                   66555 non-null object
points                       66555 non-null float64
dtypes: float64(3), int64(6), object(6)
memory usage: 8.1+ MB


In [18]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_test['country'] = lb.fit_transform(df_test['country'])
df_test['rank'] = lb.fit_transform(df_test['rank'])
df_test['level_type'] = lb.fit_transform(df_test['level_type'])

In [21]:
X_test = df_test.drop(['ID','user_id','problem_id'],axis=1).values

In [22]:
clf_KN = KNeighborsClassifier()
clf_KN.fit(X,y)
y_pred = clf_KN.predict(X_test)

In [23]:
df_sample_sub['ID'] = df_test.ID

In [24]:
df_sample_sub['attempts_range'] = y_pred

In [26]:
df_sample_sub.to_csv("Knn_without_FE.csv",index=False)